In [141]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import cv2 #画像処理のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy import signal  #信号処理のためのモジュール
from scipy.stats import skew, kurtosis  #歪度と尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.preprocessing import LabelEncoder  #カテゴリ変数を数値化するためのモジュール
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score, f1_score, fbeta_score#機械学習モデルの性能評価のためのモジュール
import tensorflow as tf #TensorFlow(Googleが開発したオープンソースの機械学習フレームワーク)
from tensorflow import keras  #TensorFlow用のニューラルネットワークライブラリAPI
from tensorflow.keras import layers #ニューラルネットワークのレイヤーを定義するためのモジュール
from keras.utils import np_utils

In [ ]:
#メモ
#画像のサイズは縦：横=1088:1920=17:30

In [142]:
#自分のGoogle Driveとドッキング
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
#GPUを使うための処理
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [144]:
#マクロの定義
N = 1 #Encoderの層の数
NUM_HEADS = 8
KEY_DIM = 30
DROPOUT = 0.1 #ドロップアウト率
LENGTH = 240
HEIGHT = 136

In [145]:
#正解データへのパスとファイル名取得
CollectPath = "/content/drive/MyDrive/DICOMO/正解データ/"
CollectFilename = os.listdir(CollectPath)

In [146]:
#不正解データへのパスとファイル名取得
IncollectPath = "/content/drive/MyDrive/DICOMO/不正解データ/"
IncollectFilename = os.listdir(IncollectPath)

In [147]:
#教師データを格納するリストを定義
img = []

In [148]:
#正解データをグレースケールに変換・リサイズ
for i in CollectFilename:
     image = cv2.imread(CollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(LENGTH, HEIGHT))
     img.append(image)

In [149]:
#不正解データをグレースケールに変換・リサイズ
for i in IncollectFilename:
     image = cv2.imread(IncollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(LENGTH, HEIGHT))
     img.append(image)

In [150]:
#教師ラベルを作成
#渡れる時を1、渡れない時を0とする
CollectLabel = np.ones(len(CollectFilename))
IncollectLabel = np.zeros(len(IncollectFilename))
label = np.concatenate((CollectLabel, IncollectLabel))

In [151]:
img = np.array(img)

In [152]:
label = np_utils.to_categorical(label)

In [154]:
#トレーニングデータとテストデータの分割
x_train, x_test, y_train, y_test = train_test_split(img, label, train_size = 0.8, shuffle = True)

In [156]:
# Define the input shape
input_shape = (HEIGHT, LENGTH)
output_shape = (1,)

#形を定義(このモジュールは行列でないとダメっぽい)
x_encoder = layers.Input(shape=input_shape)

In [157]:
#Transformer Encoder Layer(BERT)
for i in range(N):
      #Multi-Head-Attention Layer
      attention_encoder = layers.MultiHeadAttention(num_heads=NUM_HEADS, key_dim=KEY_DIM, use_bias=True)(x_encoder, x_encoder, x_encoder)

      #Dropout Layer
      attention_encoder = layers.Dropout(rate=DROPOUT)(attention_encoder)
      #Add & Norm Layer
      attention_encoder = layers.LayerNormalization()(x_encoder + attention_encoder)

      #Feed-Forward-Network
      ffn_encoder = layers.Dense(LENGTH * 4, use_bias=True, activation="relu")(attention_encoder)
      ffn_encoder = layers.Dense(LENGTH, use_bias=True)(ffn_encoder)

      #Dropout Layer
      ffn_encoder = layers.Dropout(rate=DROPOUT)(ffn_encoder)
      #Add & Norm Layer
      x_encoder = layers.LayerNormalization()(attention_encoder + ffn_encoder)

In [158]:
x = layers.Flatten()(x_encoder)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

In [159]:
model = keras.Model(inputs=x_encoder, outputs=outputs)

In [160]:
# compile model
model.compile(
    loss="categorical_crossentropy",   # 誤差関数
    optimizer="adam",     # 最適化手法
    metrics=['accuracy']
    )

In [161]:
model.fit(x_train, y_train, batch_size=32, epochs=50, shuffle=True, validation_split=0.2)

Epoch 1/50
13/13 [==============================] - 1s 20ms/step - loss: 2660.1548 - accuracy: 0.5013 - val_loss: 1625.2693 - val_accuracy: 0.5155
Epoch 2/50
13/13 [==============================] - 0s 8ms/step - loss: 715.3195 - accuracy: 0.6331 - val_loss: 526.2020 - val_accuracy: 0.5773
Epoch 3/50
13/13 [==============================] - 0s 8ms/step - loss: 328.0065 - accuracy: 0.7080 - val_loss: 331.1321 - val_accuracy: 0.8351
Epoch 4/50
13/13 [==============================] - 0s 9ms/step - loss: 290.0436 - accuracy: 0.7416 - val_loss: 226.6278 - val_accuracy: 0.6804
Epoch 5/50
13/13 [==============================] - 0s 8ms/step - loss: 299.7168 - accuracy: 0.7132 - val_loss: 127.7453 - val_accuracy: 0.9072
Epoch 6/50
13/13 [==============================] - 0s 7ms/step - loss: 129.1030 - accuracy: 0.8295 - val_loss: 170.5486 - val_accuracy: 0.7216
Epoch 7/50
13/13 [==============================] - 0s 8ms/step - loss: 126.2223 - accuracy: 0.8114 - val_loss: 138.1709 - val_accura

In [167]:
y_pred = model.predict(x_test)

4/4 [==============================] - 0s 5ms/step


TypeError: ignored

In [162]:
accuracy_score(y_test, y_pred)

ValueError: ignored

In [164]:
y_pred

array([[0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0578864e-18, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [5.8395087e-11, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.

In [165]:
y_test

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.

In [ ]:
y_pred = np.array(y_pred)